# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
browser = webdriver.Chrome(executable_path='../Tool/chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 2

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#

last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        print('網頁已達最底端')
        break
    else:
        print('網頁更新中....')
        last_height = new_height
    
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''


網頁更新中....
網頁更新中....
網頁更新中....
網頁更新中....
網頁更新中....
網頁更新中....
網頁更新中....
網頁更新中....
網頁更新中....
網頁已達最底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [4]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [5]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 大樂透加碼最終期25組100萬全送出！　1張50元挑戰歷年最高獎金

▲大樂透端午加碼百組百萬進入最後加開，2500萬獎金將一次送出。（示意圖／記者鄒鎮宇攝）
記者紀佳妘／台北報導
大樂透端午加碼100組100萬元，今（10）日將加開最後25組，2,500萬獎金將一次送出，而這次為歷年大樂透「100組100萬元」加碼，最後一期剩餘100萬獎項組數最多的一次，也就是說民眾抱走均分後的大樂透加碼100萬元獎項，有可能是歷年大樂透加碼「100組100萬元」最高的獎金。
今天是大樂透端午加開「100組100萬元」獎項的最後一期，會將25組100萬元，也就是總獎金2,500萬元全數送出。台灣彩券公司預估，若當期大樂透的銷售金額介於1.3億元至1.5億元之間，依機率計算25組100萬元獎項約開出4到5組，若皆為一注獨得，最後一期對中加碼100萬元獎項的民眾，有機會抱走約500萬至625萬元的獎金，為原獎金的5至6倍之多。
觀察今年抱走大樂透端午加開獎項的幸運兒中，有8位民眾採用「包牌」的方式幸運抱走加碼獎金，其中以花1,400元，包牌8個號碼，最受財神爺的眷顧；從機率角度來看，大樂透包牌8個號碼，相當於將對中頭獎或是加開100萬元獎項的機率提高為原本的28倍。
台彩指出，若幸運對中當期頭獎的6個獎號（不含特別號），不但可以抱走億元頭獎獎金，還可以再抱走12注參獎以及15注伍獎。
若包牌的8個號碼同時對中6個獎號和特別號，則將一舉囊括1注頭獎、6注貳獎、6注參獎和15注肆獎。台彩表示，民眾不妨把握難得的好機會，揪親友集資包牌、拚大獎。
----------------------------------------------------------------------

[1] 大同深夜回應　就經濟部否准登記將依法提起訴願

▲大同股份有限公司就經濟部之否准登記將依法提起訴願。（圖／記者湯興漢攝）
記者陳心怡／台北報導
經濟部商業司日前宣布，不准予大同公司董事變更登記。對此，大同公司昨（9）日深夜發佈公告回應，指大同全無侵害其他合法股東平等及積極行使股東權利之情事；經濟部否准後，該公司就經濟部之否准登記將依法提起訴願。
大同